In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold,train_test_split
import gc

In [2]:
label_address = "./data/label.csv"
test_address = "./data/test_set.csv"
train_address = "./data/train_set.csv"
orders_address = "./data/orders.csv"

In [3]:
train = pd.read_csv(train_address, index_col=0)
label = pd.read_csv(label_address, index_col=0)
test = pd.read_csv(test_address, index_col=0)
orders = pd.read_csv(orders_address)

/Users/yuki/.local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train

,order_id,product_id,user_id,user_orders_num,user_history_item_num,user_history_different_item_num,user_order_frequency_bydays,user_average_order,order_hour_of_day,days_since_prior_order,day_weight,aisle_id,department_id,product_ordered_num,product_reordered_num,product_reordered_rate,userproduct_order_num,userproduct_order_rate,userproduct_last_order,userproduct_reorder_rate
0,1187899,17122,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,24.0,4.0,1795.0,752.0,0.418942,21.0,1.909091,3381244.0,163.181820
1,1187899,196,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,26.0,7.0,15.0,5.0,0.333333,22.0,2.000000,3172718.0,1.363636
2,1187899,26405,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,41.0,8.0,5.0,1.0,0.200000,27.0,2.454545,3419510.0,0.454545
3,1187899,46149,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,31.0,7.0,172.0,76.0,0.441860,8.0,0.727273,3379130.0,15.636364
4,1187899,14084,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,50.0,19.0,37.0,15.0,0.405405,20.0,1.818182,3186470.0,3.363636
5,1187899,13032,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,121.0,14.0,125.0,73.0,0.584000,8.0,0.727273,2994523.0,11.363636
6,1187899,26088,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,116.0,1.0,19.0,2.0,0.105263,12.0,1.090909,3407471.0,1.727273
7,1187899,39657,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,124.0,5.0,77.0,53.0,0.688312,18.0,1.636364,3015696.0,7.000000
8,1187899,12427,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,116.0,1.0,27.0,12.0,0.444444,26.0,2.363636,3401791.0,2.454546
9,1187899,25133,1,11,59,18,19.000000,5.363637,8,14.0,0.736842,100.0,21.0,540.0,266.0,0.492593,10.0,0.909091,3378651.0,49.090908


In [5]:
label

,label
0,False
1,True
2,True
3,True
4,False
5,True
6,True
7,True
8,False
9,True


In [6]:
test

,order_id,product_id,user_id,user_orders_num,user_history_item_num,user_history_different_item_num,user_order_frequency_bydays,user_average_order,order_hour_of_day,days_since_prior_order,day_weight,aisle_id,department_id,product_ordered_num,product_reordered_num,product_reordered_rate,userproduct_order_num,userproduct_order_rate,userproduct_last_order,userproduct_reorder_rate
0,2774568,17668,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,55.0,11.0,90.0,36.0,0.400000,8.0,0.615385,2465401.0,6.923077
1,2774568,44683,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,116.0,1.0,269.0,113.0,0.420074,4.0,0.307692,2506956.0,20.692308
2,2774568,48523,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,99.0,15.0,509.0,280.0,0.550098,17.0,1.307692,3371328.0,39.153847
3,2774568,21903,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,20.0,11.0,25.0,6.0,0.240000,7.0,0.538462,2953208.0,1.923077
4,2774568,14992,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,79.0,1.0,16.0,8.0,0.500000,46.0,3.538462,3387174.0,1.230769
5,2774568,21137,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,89.0,13.0,19.0,4.0,0.210526,54.0,4.153846,3343155.0,1.461538
6,2774568,32402,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,84.0,16.0,9130.0,7434.0,0.814239,9.0,0.692308,3285305.0,702.307700
7,2774568,22035,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,29.0,13.0,24.0,2.0,0.083333,19.0,1.461538,3375757.0,1.846154
8,2774568,49683,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,124.0,5.0,9.0,1.0,0.111111,12.0,0.923077,3210069.0,0.692308
9,2774568,39190,3,13,88,33,12.000000,6.769231,15,11.0,0.916667,30.0,6.0,39.0,4.0,0.102564,11.0,0.846154,3416902.0,3.000000


In [7]:
orders.head()
test2 = orders[orders.eval_set =='test']
label["label"] = label["label"].astype(np.float32)
label = label["label"].values.tolist()

In [8]:
label

[0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [9]:
#X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.33, random_state=7)
x_train = train
x_test = label
y_train = test




In [10]:
model = XGBClassifier()

In [11]:
gc.collect()
model.fit(x_train, x_test)

[23:18:27] WARNING: /Users/travis/build/dmlc/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='approx', validate_parameters=1, verbosity=None)

In [12]:
#print(x_train.shape)
#print(x_test.shape)
#print(y_train.shape)
#print(y_train.head())

In [13]:
y_pred = model.predict(y_train)

In [14]:
predictions = [value for value in y_pred]
#print(sum(predictions))
#print(predictions[-1])

In [15]:
test["result"] = predictions

In [16]:
#print(sum(predictions))

In [17]:
#y_test.label.value_counts()

In [18]:
test2 = orders[orders.eval_set =='test']


In [21]:
TRESHOLD = 0.22 
d = dict()
for row in test.itertuples():
    if row.result > TRESHOLD:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)
for order in test2.order_id:
    if order not in d:
        d[order] = 'None'
        
submission = pd.DataFrame.from_dict(d, orient='index')

submission.reset_index(inplace=True)
submission.columns = ['order_id', 'products']
submission.to_csv('submission_original_xgboost.csv', index=False)

In [22]:
ligbm = pd.read_csv("./submission_original_xgboost.csv", index_col=0)
